In [1]:
import numpy as np
import torch
import math

## Computer quantile for kl_loss_corner

In [4]:
def compute_score(data_path):
    data = np.load(data_path, allow_pickle=True)
    tp = data.item()['tp']
    scores = []
    for i in range(len(tp)):
        prediction = np.array(tp[i][0])
        pred = prediction[:,:8]
        target = np.array(tp[i][1])
        cov = prediction[:,9:]
        score = np.abs(pred-target) / np.sqrt(np.exp(cov))
        for s in score:
            scores.append(list(s))
    return np.array(scores)

def compute_quantile(scores, alpha):
    n = scores.shape[0]
    q_level = np.ceil((n+1)*(1-alpha))/n
    quantile = []
    for i in range(scores.shape[1]):
        q = np.quantile(scores[:,i], q_level, interpolation='higher')
        quantile.append(q)
    return quantile

### disco

In [3]:
data_path = "check/check_loss_two_step_corner_val/disco/no_rsu/match_all_data_100_5.npy"
confident_range = 0.6827
scores = compute_score(data_path)
print(scores.shape)
quantile = compute_quantile(scores, 1-confident_range)
print(quantile)

(41598, 8)
[5.685563231556715, 5.7333907000666064, 5.8156107496472655, 5.698934531905109, 5.349435233304022, 5.286866601436044, 5.364550922522339, 5.254597136256393]


In [10]:
data_path = "check/check_loss_two_step_corner_val/disco/no_rsu/match_all_data_100_5.npy"
confident_range = 0.9545
scores = compute_score(data_path)
quantile = compute_quantile(scores, 1-confident_range)
print(quantile)

[17.925578517687292, 16.902509757000026, 17.921683447998383, 16.65845903704566, 15.847321990328444, 13.83974749789184, 15.933211831132338, 13.469367139403102]


In [11]:
data_path = "check/check_loss_two_step_corner_val/disco/no_rsu/match_all_data_100_5.npy"
confident_range = 0.9973
scores = compute_score(data_path)
quantile = compute_quantile(scores, 1-confident_range)
print(quantile)

[48.667429272195506, 47.12830691785341, 47.89193980424878, 42.536190890924544, 42.169949853660526, 46.371733470171094, 41.253330827583945, 41.55867648053199]


### upperbound

In [5]:
data_path = "check/check_loss_two_step_corner_val/upperbound/no_rsu/match_all_data_100_5.npy"
confident_range = 0.6827
scores = compute_score(data_path)
quantile = compute_quantile(scores, 1-confident_range)
print(quantile)

[4.190345185940821, 4.351993435985297, 4.558479310459919, 4.717549387208282, 4.278715029642637, 4.102446480783583, 4.1725601258012475, 3.530299361843318]


In [12]:
data_path = "check/check_loss_two_step_corner_val/upperbound/no_rsu/match_all_data_100_5.npy"
confident_range = 0.9973
scores = compute_score(data_path)
quantile = compute_quantile(scores, 1-confident_range)
print(quantile)

[49.72913250209128, 68.07916849224074, 54.76645263747507, 76.2931006003976, 46.32874805418221, 76.94147233490142, 50.70841262977031, 77.77131089570797]


### lowerbound

In [6]:
data_path = "check/check_loss_two_step_corner_val/lowerbound/no_rsu/match_all_data_100_5.npy"
confident_range = 0.6827
scores = compute_score(data_path)
quantile = compute_quantile(scores, 1-confident_range)
print(quantile)

[6.10302951960435, 6.182331226887726, 5.740435199991636, 6.236356538776135, 6.334263986132204, 5.1828751359733305, 6.399557006521807, 5.073939114533967]


In [13]:
data_path = "check/check_loss_two_step_corner_val/lowerbound/no_rsu/match_all_data_100_5.npy"
confident_range = 0.9973
scores = compute_score(data_path)
quantile = compute_quantile(scores, 1-confident_range)
print(quantile)

[259.92825137969436, 201.07436055964888, 258.2222934053669, 197.05243747520606, 328.5189148613409, 235.0538877505854, 339.6938946129562, 235.97804342045072]


## Computer quantile for kl_loss_center_ind

In [23]:
def corner_to_center_box2d_torch(corners):
    p1 = corners[:,0:2]
    p2 = corners[:,2:4]
    p3 = corners[:,4:6]
    p4 = corners[:,6:8]
    center = (p1+p2+p3+p4)/4
    w = (np.sqrt(np.sum(np.power(p1-p4, 2), axis=1, keepdims = True)) + np.sqrt(np.sum(np.power(p2-p3, 2), axis=1, keepdims = True)))/2
    h = (np.sqrt(np.sum(np.power(p1-p2, 2), axis=1, keepdims = True)) + np.sqrt(np.sum(np.power(p3-p4, 2), axis=1, keepdims = True)))/2
    wp = np.sqrt(np.sum(np.power(p1-p4, 2), axis=1, keepdims = True))
    sina = np.divide((p4[:,1:2] - p1[:,1:2]), wp)
    cosa = np.divide((p1[:,0:1] - p4[:,0:1]), wp)
    result = np.concatenate((center, w, h, sina, cosa), axis=1)
    return result

def compute_score_center(data_path):
    data = np.load(data_path, allow_pickle=True)
    tp = data.item()['tp']
    scores = []
    for i in range(len(tp)):
        prediction = np.array(tp[i][0])
        pred = prediction[:,:8]
        target = np.array(tp[i][1])
        cov = prediction[:,9:]
        std = np.sqrt(np.exp(cov))
        pred = corner_to_center_box2d_torch(pred)
        target = corner_to_center_box2d_torch(target)
        score = np.abs(pred-target) / np.sqrt(np.exp(cov))
        for s in score:
            scores.append(list(s))
    return np.array(scores)

def compute_quantile_center(scores, alpha):
    n = scores.shape[0]
    q_level = np.ceil((n+1)*(1-alpha))/n
    quantile = []
    for i in range(scores.shape[1]):
        q = np.quantile(scores[:,i], q_level, interpolation='higher')
        quantile.append(q)
    return quantile

### disco

In [24]:
data_path = "check/check_loss_two_step_center_ind_val/disco/no_rsu/match_all_data_100_5.npy"
confident_range = 0.6827
scores = compute_score_center(data_path)
quantile = compute_quantile_center(scores, 1-confident_range)
print(quantile)

[5.468984395243384, 4.89869516767041, 16.09847077479864, 3.0882512529684347, 1.0243907317028713, 4.811753752332904]


In [16]:
data_path = "check/check_loss_two_step_center_ind_val/disco/no_rsu/match_all_data_100_5.npy"
confident_range = 0.9973
scores = compute_score_center(data_path)
quantile = compute_quantile_center(scores, 1-confident_range)
print(quantile)

[48.40839894811819, 25.026924203602636, 131.90336169825727, 15.255118427560566, 59.67641819394204, 71.02182939111708]


### upperbound

In [17]:
data_path = "check/check_loss_two_step_center_ind_val/upperbound/no_rsu/match_all_data_100_5.npy"
confident_range = 0.6827
scores = compute_score_center(data_path)
quantile = compute_quantile_center(scores, 1-confident_range)
print(quantile)

[3.9270005592398936, 3.7293620083399235, 12.661970846621758, 3.507125215592104, 1.3240707560700242, 3.309770337192223]


In [18]:
data_path = "check/check_loss_two_step_center_ind_val/upperbound/no_rsu/match_all_data_100_5.npy"
confident_range = 0.9973
scores = compute_score_center(data_path)
quantile = compute_quantile_center(scores, 1-confident_range)
print(quantile)

[31.340185602346363, 24.276376908593676, 118.64887029003187, 18.253235606033712, 109.68835650542137, 56.51205738298187]


### lowerbound

In [19]:
data_path = "check/check_loss_two_step_center_ind_val/lowerbound/no_rsu/match_all_data_100_5.npy"
confident_range = 0.6827
scores = compute_score_center(data_path)
quantile = compute_quantile_center(scores, 1-confident_range)
print(quantile)

[4.824761883365095, 4.435752094000069, 14.379824436595687, 2.8108565130701026, 1.0106651376400175, 4.404153510782447]


In [20]:
data_path = "check/check_loss_two_step_center_ind_val/lowerbound/no_rsu/match_all_data_100_5.npy"
confident_range = 0.9973
scores = compute_score_center(data_path)
quantile = compute_quantile_center(scores, 1-confident_range)
print(quantile)

[44.646381707137465, 28.842831596825288, 127.50113292712857, 18.538402799100826, 81.66889262783303, 53.23585004452376]


In [2]:
math.sqrt(math.exp(-6.51))

0.03858082066337404

## Computer quantile for kl_loss_center_sr_ind

In [6]:
def corner_to_center_box2d_torch(corners):
    p1 = corners[:,0:2]
    p2 = corners[:,2:4]
    p3 = corners[:,4:6]
    p4 = corners[:,6:8]
    center = (p1+p2+p3+p4)/4
    w = (np.sqrt(np.sum(np.power(p1-p4, 2), axis=1, keepdims = True)) + np.sqrt(np.sum(np.power(p2-p3, 2), axis=1, keepdims = True)))/2
    h = (np.sqrt(np.sum(np.power(p1-p2, 2), axis=1, keepdims = True)) + np.sqrt(np.sum(np.power(p3-p4, 2), axis=1, keepdims = True)))/2
    s = np.multiply(w,h)
    r = np.divide(w,h)
    wp = np.sqrt(np.sum(np.power(p1-p4, 2), axis=1, keepdims = True))
    sina = np.divide((p4[:,1:2] - p1[:,1:2]), wp)
    cosa = np.divide((p1[:,0:1] - p4[:,0:1]), wp)
    result = np.concatenate((center, s, r, sina, cosa), axis=1)
    return result

def compute_score_center(data_path):
    data = np.load(data_path, allow_pickle=True)
    tp = data.item()['tp']
    scores = []
    for i in range(len(tp)):
        prediction = np.array(tp[i][0])
        pred = prediction[:,:8]
        target = np.array(tp[i][1])
        cov = prediction[:,9:]
        std = np.sqrt(np.exp(cov))
        pred = corner_to_center_box2d_torch(pred)
        target = corner_to_center_box2d_torch(target)
        score = np.abs(pred-target) / np.sqrt(np.exp(cov))
        for s in score:
            scores.append(list(s))
    return np.array(scores)

def compute_quantile_center(scores, alpha):
    n = scores.shape[0]
    q_level = np.ceil((n+1)*(1-alpha))/n
    quantile = []
    for i in range(scores.shape[1]):
        q = np.quantile(scores[:,i], q_level, interpolation='higher')
        quantile.append(q)
    return quantile

### disco

In [7]:
data_path = "check/check_loss_two_step_center_sr_ind_val/disco/no_rsu/match_all_data_100_5.npy"
confident_range = 0.6827
scores = compute_score_center(data_path)
quantile = compute_quantile_center(scores, 1-confident_range)
print(quantile)

[4.737979515372718, 4.714078400210938, 7.567237656129541, 34.6806279071411, 0.8099997963207344, 4.143097399364225]


In [8]:
data_path = "check/check_loss_two_step_center_sr_ind_val/disco/no_rsu/match_all_data_100_5.npy"
confident_range = 0.9973
scores = compute_score_center(data_path)
quantile = compute_quantile_center(scores, 1-confident_range)
print(quantile)

[49.07599482807287, 24.25584494535801, 43.96985950550828, 192.27719839830377, 48.24540340451742, 63.485555875830585]


In [2]:
a = np.array([49.07599482807287, 24.25584494535801, 43.96985950550828, 192.27719839830377]) / 3
b = a * a
print(list(b))
print(list(a))

[267.6059187072261, 65.37177933480551, 214.8165049926819, 4107.835669322297]
[16.358664942690957, 8.08528164845267, 14.656619835169428, 64.09239946610126]


### upperbound

In [9]:
data_path = "check/check_loss_two_step_center_sr_ind_val/upperbound/no_rsu/match_all_data_100_5.npy"
confident_range = 0.6827
scores = compute_score_center(data_path)
quantile = compute_quantile_center(scores, 1-confident_range)
print(quantile)

[3.818482185248409, 3.9026028441206817, 6.644751592651442, 33.49078846043044, 1.150774290024538, 3.541034592403047]


In [10]:
data_path = "check/check_loss_two_step_center_sr_ind_val/upperbound/no_rsu/match_all_data_100_5.npy"
confident_range = 0.9973
scores = compute_score_center(data_path)
quantile = compute_quantile_center(scores, 1-confident_range)
print(quantile)

[33.60120205831025, 22.88242090727398, 44.92134567665782, 187.53907271555377, 83.97342833677149, 57.77635661409111]


In [3]:
a = np.array([33.60120205831025, 22.88242090727398, 44.92134567665782, 187.53907271555377]) / 3
b = a * a
print(list(b))
print(list(a))

[125.44897552926587, 58.17835406418326, 224.21414415575381, 3907.8781994455294]
[11.200400686103416, 7.627473635757993, 14.973781892219273, 62.51302423851792]


### lowerbound

In [11]:
data_path = "check/check_loss_two_step_center_sr_ind_val/lowerbound/no_rsu/match_all_data_100_5.npy"
confident_range = 0.6827
scores = compute_score_center(data_path)
quantile = compute_quantile_center(scores, 1-confident_range)
print(quantile)

[4.815796820342549, 4.617622610425134, 7.233958697265688, 32.38028565917383, 1.3595152977293166, 3.4976893395704143]


In [12]:
data_path = "check/check_loss_two_step_center_sr_ind_val/lowerbound/no_rsu/match_all_data_100_5.npy"
confident_range = 0.9973
scores = compute_score_center(data_path)
quantile = compute_quantile_center(scores, 1-confident_range)
print(quantile)

[55.27816652981975, 31.64422204339833, 51.81092885206998, 368.3591724482785, 94.98653035446952, 40.695949853779744]


In [6]:
a = np.array([55.27816652981975, 31.64422204339833, 51.81092885206998, 368.3591724482785]) / 3
a = a * a
print(list(a))

[339.5195216553872, 111.26186541465519, 298.26359427936194, 15076.497769642287]


In [7]:
np.array([55.27816652981975, 31.64422204339833, 51.81092885206998, 368.3591724482785]) / 3

array([ 18.42605551,  10.54807401,  17.27030962, 122.78639082])